# 06 - Embedded Multi Layer LSTM on our preprocessed text

Let's feed the generated data from the previous notebook into our LSTM from chapter 4!

This time we load our rap text from the **prepped** directory

In [1]:
import tools.architectures as nn
import tools.processing as pre
import tools.training as tr

text = pre.get_text("data/prepped/clean2_pac.txt")[:60000]
vocab = pre.Vocabulary(text)

text2 = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")[:60000]
vocab2 = pre.Vocabulary(text2)

### The prepared text should have reduced number of words

In [2]:
print( f"vocab size in prepped text: \t{len(vocab.index2word)}")
print( f"vocab size in previous text: \t{len(vocab2.index2word)}")

vocab size in prepped text: 	1938
vocab size in previous text: 	2181


In [3]:
TIMESTEPS = 20

vocab = pre.Vocabulary(text)

tokens = text.split(" ")[:-1]

str_data, str_labels = pre.create_data_label_pairs(tokens, TIMESTEPS)

print( list( zip(str_data, str_labels) )[:5] )

[(['as', 'real', 'as', 'it', 'seems', 'the', 'american', 'dream', ';', 'is', 'not', 'nothing', 'but', 'another', 'calculated', 'schemes', ';', 'to', 'get', 'us'], 'locked'), (['real', 'as', 'it', 'seems', 'the', 'american', 'dream', ';', 'is', 'not', 'nothing', 'but', 'another', 'calculated', 'schemes', ';', 'to', 'get', 'us', 'locked'], 'up'), (['as', 'it', 'seems', 'the', 'american', 'dream', ';', 'is', 'not', 'nothing', 'but', 'another', 'calculated', 'schemes', ';', 'to', 'get', 'us', 'locked', 'up'], 'shot'), (['it', 'seems', 'the', 'american', 'dream', ';', 'is', 'not', 'nothing', 'but', 'another', 'calculated', 'schemes', ';', 'to', 'get', 'us', 'locked', 'up', 'shot'], 'up'), (['seems', 'the', 'american', 'dream', ';', 'is', 'not', 'nothing', 'but', 'another', 'calculated', 'schemes', ';', 'to', 'get', 'us', 'locked', 'up', 'shot', 'up'], 'back')]


In [4]:
encoder = tr.IndexWordEncoder("Index-Word-Encoding", vocab.word2index)
decoder = tr.OneHotWordDecoder("1-Hot-Word-Decoding", vocab.index2word, temperature=0.5)

data = encoder.encode( str_data )
labels = encoder.encode_labels( str_labels )

my_seed = " ".join(str_data[0]).replace("\\n", "\n")

del str_data, str_labels

In [5]:
HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = len(vocab.word2index)

EPOCHS = 15
BATCH_SIZE = 256

EMBEDDING_SIZE = 256

rnn = nn.EmbeddedSingleLayerRNN(name = "multi-2pac")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, EMBEDDING_SIZE, TIMESTEPS, l2_reg=0.0)

sampler = lambda trainable, _: tr.sample( my_seed, trainable, encoder, decoder, length=50)

tr.train_model(rnn, data, labels, sampler, epochs=EPOCHS, batch_size=BATCH_SIZE, log_dir="logs/06/2pac")

Building model from scratch! 
 Saving into: 'logs/06/testing'


Epoch 1/15
Loss:    	 5.391435623168945
Accuracy:	 15.765629768371582
------Sampling----------
seed: 
as real as it seems the american dream ; is not nothing but another calculated schemes ; to get us
-
result: 
as real as it seems the american dream ; is not nothing but another calculated schemes ; to get us the ; i am i am now ; i am the on a nigga ; and i am the ; i am the the ; i am the to i am the am the when ; he is ; i am and like and the to ; the monday the


Epoch 2/15
Loss:    	 4.9652838706970215
Accuracy:	 18.838720321655273
------Sampling----------
seed: 
as real as it seems the american dream ; is not nothing but another calculated schemes ; to get us
-
result: 
as real as it seems the american dream ; is not nothing but another calculated schemes ; to get us to the ready ; i am a smooth america ; but i do not the nigga ; so i am it is a police ; i do not a to get ; i am the facts ; the nigga i am to the keep

In [7]:
decoder.temperature = 0.8

sampler = lambda seed_text: tr.sample( seed_text, rnn, encoder, decoder, length=50)

sampler("when i was thirteen ; i had my first love ; there was nobody that could put hands on my baby ; and nobody came between us that could ever come above")

------Sampling----------
seed: 
when i was thirteen ; i had my first love ; there was nobody that could put hands on my baby ; and nobody came between us that could ever come above
-
result: 
when i was thirteen ; i had my first love ; there was nobody that could put hands on my baby ; and nobody came between us that could ever come above ; but now i am guilty of falling for his girlfriend ; i know it is wrong but it is not a cop damn ; i am trapped in the hell ; one if you will find out my homies ; it is the nigga that you the nigga ;
